In [1]:
import enum
import copy
import itertools
import functools
import collections

import numpy as np

import utility as util

In [39]:
class IDMaker(object):
    """Create ID string from ID primitives. For example when constructed via
    
    ```
    carla_id_maker = IDMaker(
        'map_name/episode/agent/frame',
        prefixes={
            'episode':  'ep',
            'agent':    'agent',
            'frame':    'frame'},
        format_spec={
            'episode':  '03d',
            'agent':    '03d',
            'frame':    '08d'})
    ```
    
    Then calling
    
    ```
    carla_id_maker.make_id(map_name='Town04', episode=1, agent=123, frame=1000)
    ```
    
    gives

    Town04/ep001/agent123/frame00001000
    """

    @staticmethod
    def __clamp(s):
        return "{" + str(s) + "}"

    def make_fstring(self):
        def f(w):
            s = self.__clamp(f"{w}:{self.__format_spec[w]}")
            return self.__prefixes[w] + s
        l = map(f, self.__sample_pattern_lst)
        return '/'.join(l)

    def __init__(self, s, prefixes={}, format_spec={}):
        self.__sample_pattern_str = s
        self.__sample_pattern_lst = s.split('/')
        self.__sample_pattern = util.create_sample_pattern(s)
        self.__prefixes = prefixes
        self.__format_spec = format_spec
        for w in self.__sample_pattern_lst:
            if w not in self.__prefixes:
                self.__prefixes[w] = ''
            if w not in self.__format_spec:
                self.__format_spec[w] = ''
        self.__fstring = self.make_fstring()

    @property
    def sample_pattern(self):
        return self.__sample_pattern
    
    @property
    def fstring(self):
        return self.__fstring
        
    def make_id(self, **kwargs):
        return self.__fstring.format(**kwargs)

    def filter_ids(self, filter, ids, inclusive=True):
        for word in filter.keys():
            if not isinstance(filter[word], (list, np.ndarray)):
                filter[word] = [filter[word]]
            
        sp = self.__sample_pattern
        id_nd = np.array([[*id.split('/'), id] for id in ids])
        common_words = set(sp) & set(filter)
        b_nd = np.zeros((len(ids), len(common_words)), dtype=bool)
        for idx, word in enumerate(common_words):
            values = filter[word]
            wd_nd = id_nd[:, sp[word]]
            f = lambda v: wd_nd == v 
            wd_nd_b = util.map_to_ndarray(f, values)
            b_nd[:, idx] = np.any(wd_nd_b, axis=0)
        if inclusive:
            b_nd = np.all(b_nd, axis=1)
        else:
            b_nd = np.any(b_nd, axis=1)
            b_nd = np.logical_not(b_nd)
        id_nd = id_nd[b_nd, -1]
        return id_nd.tolist()

carla_id_maker = IDMaker(
        'map_name/episode/agent/frame',
        prefixes={
            'episode':  'ep',
            'agent':    'agent',
            'frame':    'frame'},
        format_spec={
            'episode':  '03d',
            'agent':    '03d',
            'frame':    '08d'})

carla_id_maker.sample_pattern, carla_id_maker.fstring, \
        carla_id_maker.make_id(map_name='Town04', episode=1, agent=123, frame=1000)

({'map_name': 0, 'episode': 1, 'agent': 2, 'frame': 3},
 '{map_name:}/ep{episode:03d}/agent{agent:03d}/frame{frame:08d}',
 'Town04/ep001/agent123/frame00001000')

In [40]:
ids = ['Town01/ep001/agent001/frame00001000',
        'Town01/ep001/agent002/frame00001000',
        'Town01/ep001/agent003/frame00001000',
        'Town01/ep001/agent001/frame00002000',
        'Town01/ep001/agent002/frame00002000',
        'Town01/ep002/agent001/frame00000500',
        'Town01/ep002/agent002/frame00000500',
        'Town01/ep002/agent003/frame00000530',
        'Town02/ep002/agent004/frame00000530',
        'Town02/ep003/agent001/frame00000100',
        'Town02/ep003/agent002/frame00000100',
        'Town02/ep004/agent001/frame00000100',
        'Town02/ep004/agent002/frame00000100',
        'Town03/ep005/agent001/frame00001000',
        'Town03/ep005/agent001/frame00002000',]

filter = {'map_name': 'Town02', 'episode': 'ep003'}

carla_id_maker.filter_ids(filter, ids, inclusive=True)

[[False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False  True]
 [ True  True]
 [ True  True]
 [False  True]
 [False  True]
 [False False]
 [False False]]


['Town02/ep003/agent001/frame00000100', 'Town02/ep003/agent002/frame00000100']

In [2]:
np.zeros((2,2), dtype=bool)

array([[False, False],
       [False, False]])

In [9]:
d1 = dict(zip('abc', [1,2,3]))
d2 = dict(zip('bcd', [1,2,3]))

set(d1) & set(d2), list(set(d1) & set(d2)), [v for v in set(d1) & set(d2)]

({'b', 'c'}, ['b', 'c'], ['b', 'c'])

In [34]:
isinstance('asdf', (list, np.ndarray))

False